### Implementing **Eddy Elfenbein's** Two Articles on **Crossing Wall Street Website**.

In these articles Eddy show why [return-on-equity (ROE)](http://www.crossingwallstreet.com/archives/2012/09/why-return-on-equity-is-so-important.html) is so important and why [EV/EBITDA](http://www.crossingwallstreet.com/archives/2014/02/the-single-best-metric-evebitda.html) is the single most important metric

I used selenium and chromedriver to scrape data from yahoo finance to implement these concepts shared in the articles. To try to run this, you just need to add one more stock tickers in the fuction.

**Note:** chromedriver will have to to be downloaded and saved in your directory for this code to run

In [2]:
def ticks(symbol):
    import pandas as pd
    tickers = list(symbol.split(" ")) 
    from selenium import webdriver
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.common.exceptions import TimeoutException
    l=[]
    browser = webdriver.Chrome()
    browser.set_page_load_timeout(120)
    browser_buy = webdriver.Chrome()
    browser_buy.set_page_load_timeout(120)
    y=[]
    for ticker in tickers:
        site= 'https://finance.yahoo.com/quote/' +ticker+ '/analysis/'
        browser.get(site)
        site_buy= 'https://finance.yahoo.com/quote/' +ticker+ '/key-statistics/'
        browser_buy.get(site_buy)
        # get all the financial values
        e={}
        e['name']=ticker
        try:
            e['price'] = browser_buy.find_element_by_xpath('//*[@id="quote-header-info"]/div[3]/div[1]/div/span[1]').text
        except:
            pass
        try:
            e['eps_y1'] = browser.find_element_by_xpath('//*[@id="Col1-0-AnalystLeafPage-Proxy"]/section/table[1]/tbody/tr[2]/td[5]/span').text
        except:
            pass
        try:
            e['eps_growth_5y'] = browser.find_element_by_xpath('//*[@id="Col1-0-AnalystLeafPage-Proxy"]/section/table[6]/tbody/tr[5]/td[2]').text
        except:
            pass
        try:
            e['ev_ebitda'] = browser_buy.find_element_by_xpath('//*[@id="Col1-0-KeyStatistics-Proxy"]/section/div[2]/div[1]/div/div/div[1]/table/tbody/tr[9]/td[2]').text
        except:
            pass
        try:
            e['roe'] = browser_buy.find_element_by_xpath('//*[@id="Col1-0-KeyStatistics-Proxy"]/section/div[2]/div[3]/div/div[3]/div/table/tbody/tr[2]/td[2]').text
        except:
            pass
        try:
            e['pe'] = browser_buy.find_element_by_xpath('//*[@id="Col1-0-KeyStatistics-Proxy"]/section/div[2]/div[1]/div/div/div[1]/table/tbody/tr[3]/td[2]').text
        except:
            pass
        try:
            e['peg'] = browser_buy.find_element_by_xpath('//*[@id="Col1-0-KeyStatistics-Proxy"]/section/div[2]/div[1]/div/div/div[1]/table/tbody/tr[5]/td[2]').text
        except:
            pass
        try:
            e['ps'] = browser_buy.find_element_by_xpath('//*[@id="Col1-0-KeyStatistics-Proxy"]/section/div[2]/div[1]/div/div/div[1]/table/tbody/tr[6]/td[2]').text
        except:
            pass
        y.append(e)
    #save to dataframe
    import pandas as pd
    buy_yahoo=pd.DataFrame(y)
    buy_yahoo[buy_yahoo.columns.drop('name')] = buy_yahoo[buy_yahoo.columns.drop('name')].astype(str)
    buy_yahoo=buy_yahoo.loc[buy_yahoo['roe'] !='N/A']
    buy_yahoo=buy_yahoo.loc[buy_yahoo['eps_growth_5y'] !='nan']
    buy_yahoo['roe']=(buy_yahoo['roe'].str[:-1].astype(float))
    buy_yahoo=buy_yahoo.loc[buy_yahoo['eps_growth_5y'] !='N/A']
    buy_yahoo['eps_growth_5y']=(buy_yahoo['eps_growth_5y'].str[:-1].astype(float))
    buy_yahoo[buy_yahoo.columns.drop('name')] = buy_yahoo[buy_yahoo.columns.drop('name')].apply(pd.to_numeric, errors='coerce')
    buy_yahoo['fair_pe']=(buy_yahoo.eps_growth_5y/2)+8
    buy_yahoo['fair_price']=buy_yahoo.fair_pe*buy_yahoo.eps_y1
    buy_yahoo['discount']=((buy_yahoo.fair_price - buy_yahoo.price)/buy_yahoo.fair_price)
    return buy_yahoo
    browser.quit()
    browser_buy.quit()
ticks('AAPL MESA UAL aal dal BHF MLHR')


,name,price,eps_y1,eps_growth_5y,ev_ebitda,roe,pe,peg,ps,fair_pe,fair_price,discount
0,AAPL,297.43,14.92,9.86,17.38,55.92,25.02,2.33,5.08,12.930,192.91560,-0.541762
1,MESA,8.69,2.13,13.60,5.20,11.89,6.39,0.39,0.39,14.800,31.52400,0.724337
2,UAL,87.90,12.80,14.58,5.49,27.31,8.20,0.51,0.52,15.290,195.71200,0.550871
4,dal,58.06,7.21,11.69,6.09,32.59,8.21,0.72,0.81,13.845,99.82245,0.418367
5,BHF,38.99,9.72,13.39,2.24,11.57,2.58,0.60,0.41,14.695,142.83540,0.727028
6,MLHR,41.60,3.67,50.60,8.71,27.33,11.61,0.25,0.93,33.300,122.21100,0.659605
